In [1]:
from surprise import SVD, SVDpp
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [2]:
import pandas as pd
data = pd.read_csv('test-Copy1.csv')
data['rating'] = -1
data.to_csv('haha.csv', index = False)

In [ ]:
reader = Reader(line_format='user item rating', sep=',', skip_lines=1)
train = Dataset.load_from_file('ratings-final-processed.csv', reader=reader)
test = Dataset.load_from_file('impressions-train-processed.csv', reader=reader)
#test2 = Dataset.load_from_file('haha.csv', reader=reader)

In [4]:
algo = SVD(n_epochs = 30, lr_all = 0.005, n_factors = 50)
train, _ = train_test_split(train, test_size=.00000000001)
_, test  = train_test_split(test, test_size=.99999999999)

In [5]:
algo.fit(train)
predictions = algo.test(test)

In [6]:
accuracy.rmse(predictions, verbose=True)

RMSE: 1.1085


1.1085359498627934

In [7]:
user = []
movie = []
RealRate = []
Estimated = []
for userID, movieID, actualRating, estimatedRating, _ in predictions:
    user.append(int(userID))
    movie.append(int(movieID))
    RealRate.append(actualRating)
    Estimated.append(estimatedRating)

In [8]:
import pandas as pd
predicted = pd.DataFrame()
predicted['userId'] = user
predicted['movieId'] = movie
predicted['RealRate'] = RealRate
predicted['Estimated'] = Estimated
predicted['RealRate'] = predicted['RealRate'].astype(int)
predicted.head()

,userId,movieId,RealRate,Estimated
0,1,57,1,1.890420
1,548,150,0,1.158163
2,435,54,2,1.653258
3,547,1,2,1.520609
4,250,91,1,1.102785


In [9]:
from sklearn import preprocessing
x = predicted[['Estimated']].values.astype(float)
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(1,3))
predicted['Normalized'] = min_max_scaler.fit_transform(x).astype(int)
predicted['Estimated'] = predicted['Estimated'].astype(int)
predicted

,userId,movieId,RealRate,Estimated,Normalized
0,1,57,1,1,2
1,548,150,0,1,1
2,435,54,2,1,2
3,547,1,2,1,1
4,250,91,1,1,1
...,...,...,...,...,...
27160,89,3,2,1,2
27161,153,143,0,1,1
27162,488,146,2,1,2
27163,244,115,2,1,2


In [11]:
predicted['Estimated'].value_counts()

1    26796
2      369
Name: Estimated, dtype: int64

In [12]:
algo.predict(str(45), str(57))

Prediction(uid='45', iid='57', r_ui=None, est=1.7624696812404885, details={'was_impossible': False})